In [1]:
import numpy as np
import pandas as pd

In [2]:
rw = pd.read_csv('data/winequality-red.csv')
rw.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
print(rw.quality.value_counts())

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64


In [4]:
# Good / Poor 2진 등급으로 분류
rw['target'] = rw.quality.apply(lambda x: 1 if x >= 6 else 0)

In [5]:
# 결측치 체크 
rw.isna().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
target                  0
dtype: int64

In [6]:
X = rw.iloc[:, :-2].values
y = rw.target.values

In [7]:
from sklearn.preprocessing import StandardScaler 
# 표준화
X_std = StandardScaler().fit_transform(X)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_std, y, stratify=y, test_size=0.2, random_state=2023
)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.ensemble import VotingClassifier
# from sklearn.model_selection import GridSearchCV 
# from sklearn.metrics import accuracy_score

In [16]:
rfc = RandomForestClassifier(random_state=2023) 
rfr = RandomForestRegressor(n_estimators=100,random_state=2023)
lrc = LogisticRegression(random_state=2023)
knn = KNeighborsClassifier()
svc = SVC(random_state=2023)
svc2 = SVC(probability=True, random_state=2023)


In [17]:
rfc.fit(X_train, y_train)
rfr.fit(X_train, y_train)
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [19]:
rfc.score(X_test, y_test) ,rfr.score(X_test, y_test) ,lrc.score(X_test, y_test), svc.score(X_test, y_test),knn.score(X_test, y_test)

(0.8625, 0.512440519643628, 0.778125, 0.790625, 0.734375)

In [ ]:
print(f'RandomForestClassifier(rfc) score: {}')

In [25]:
voc = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc), ('KNN', knn)],
    voting='hard'
)
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.784375

In [22]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb.score(X_test, y_test)

0.840625

In [23]:
from lightgbm import LGBMClassifier 
lgb = LGBMClassifier()
evals = [(X_test, y_test)]
lgb.fit(X_train, y_train, eval_set=evals, eval_metric='logloss', verbose=True)

[1]	valid_0's binary_logloss: 0.653161
[2]	valid_0's binary_logloss: 0.625011
[3]	valid_0's binary_logloss: 0.59498
[4]	valid_0's binary_logloss: 0.570736
[5]	valid_0's binary_logloss: 0.548624
[6]	valid_0's binary_logloss: 0.529952
[7]	valid_0's binary_logloss: 0.517124
[8]	valid_0's binary_logloss: 0.503533
[9]	valid_0's binary_logloss: 0.491252
[10]	valid_0's binary_logloss: 0.480848
[11]	valid_0's binary_logloss: 0.470929
[12]	valid_0's binary_logloss: 0.459038
[13]	valid_0's binary_logloss: 0.450861
[14]	valid_0's binary_logloss: 0.447485
[15]	valid_0's binary_logloss: 0.440419
[16]	valid_0's binary_logloss: 0.438103
[17]	valid_0's binary_logloss: 0.434231
[18]	valid_0's binary_logloss: 0.431824
[19]	valid_0's binary_logloss: 0.429003
[20]	valid_0's binary_logloss: 0.42519
[21]	valid_0's binary_logloss: 0.42258
[22]	valid_0's binary_logloss: 0.419985
[23]	valid_0's binary_logloss: 0.417877
[24]	valid_0's binary_logloss: 0.416492
[25]	valid_0's binary_logloss: 0.412202
[26]	valid_0

c:\Users\user\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier()

In [24]:
lgb.score(X_test, y_test)

0.846875

In [26]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=2023)
dtc.fit(X_train, y_train)
dtc.score(X_test, y_test)

0.7375